# Прогнозируем задержки самолетов

In [ ]:
!pip install catboost lightgbm optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.0 MB/s eta 0:00:00


In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [2]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [3]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Month          100000 non-null  object
 1   DayofMonth     100000 non-null  object
 2   DayOfWeek      100000 non-null  object
 3   DepTime        100000 non-null  int64 
 4   UniqueCarrier  100000 non-null  object
 5   Origin         100000 non-null  object
 6   Dest           100000 non-null  object
 7   Distance       100000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 6.1+ MB


In [5]:
cat_indices = np.where(X.dtypes == 'object')[0]
cat_cols = list(X.dtypes[X.dtypes == 'object'].index)

Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [6]:
cat_features = cat_indices
len(cat_features)

6

Разобъем данные на обучение и контроль

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [8]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [9]:
# your code here
cb = CatBoostClassifier(random_seed=RANDOM_STATE)

In [11]:
cb.fit(
    Xtrain, ytrain,
    cat_features=cat_features,
    logging_level='Silent',
    plot=False)

In [12]:
cb_pred = cb.predict_proba(Xtest)[:,1]
score_cb_def = roc_auc_score(ytest, cb_pred)
score_cb_def

0.7666402289246762

Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [13]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [14]:
# your code here
lgbm = LGBMClassifier(random_state=RANDOM_STATE)
lgbm.fit(Xtrain, ytrain)

LGBMClassifier(random_state=111)

In [17]:
lgbm_pred = lgbm.predict_proba(Xtest)[:,1]
score_lgbm_def = roc_auc_score(ytest, lgbm_pred)
score_lgbm_def

0.7341149074685321

## Optuna

Выделим дополнительную валидационную выборку.

In [20]:
import optuna

In [18]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [55]:
# your code here
def objective_lgmb(trial):
    num_leaves = trial.suggest_int('num_leaves', 10, 100)
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 15)
    max_bin = trial.suggest_int('max_bin', 5, 50)
    model = LGBMClassifier(random_state=RANDOM_STATE, num_leaves=num_leaves, n_estimators=n_estimators, max_depth=max_depth, max_bin=max_bin, n_jobs=-1)
    model.fit(Xtrain_new, ytrain_new)
    pred = model.predict_proba(Xval)[:, 1]
    return roc_auc_score(yval, pred)

In [56]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgmb, n_trials=300, n_jobs=-1)

[I 2023-06-18 20:01:08,508] A new study created in memory with name: no-name-410ddf52-2498-47d3-ab3e-e035bd3af8b4
[I 2023-06-18 20:01:09,943] Trial 0 finished with value: 0.713568577200022 and parameters: {'num_leaves': 17, 'n_estimators': 191, 'max_depth': 11, 'max_bin': 18}. Best is trial 0 with value: 0.713568577200022.
[I 2023-06-18 20:01:10,095] Trial 8 finished with value: 0.7106105626024037 and parameters: {'num_leaves': 71, 'n_estimators': 49, 'max_depth': 2, 'max_bin': 17}. Best is trial 0 with value: 0.713568577200022.
[I 2023-06-18 20:01:10,199] Trial 2 finished with value: 0.7162906282051432 and parameters: {'num_leaves': 22, 'n_estimators': 218, 'max_depth': 5, 'max_bin': 34}. Best is trial 2 with value: 0.7162906282051432.
[I 2023-06-18 20:01:10,538] Trial 9 finished with value: 0.7213970755060106 and parameters: {'num_leaves': 69, 'n_estimators': 74, 'max_depth': 4, 'max_bin': 27}. Best is trial 9 with value: 0.7213970755060106.
[I 2023-06-18 20:01:10,938] Trial 5 finish

In [57]:
study.best_params

{'num_leaves': 41, 'n_estimators': 181, 'max_depth': 3, 'max_bin': 50}

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [58]:
# your code here
lgbm.set_params(**study.best_params)
lgbm.fit(Xtrain, ytrain)

LGBMClassifier(max_bin=50, max_depth=3, n_estimators=181, num_leaves=41,
               random_state=111)

In [59]:
lgbm_pred = lgbm.predict_proba(Xtest)[:,1]
score_lgbm = roc_auc_score(ytest, lgbm_pred)
print(f'opt: {score_lgbm}\ndef: {score_lgbm_def}')

opt: 0.732418702370703
def: 0.7341149074685321


## Quiz

Чему равно количество листьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!